In [26]:
import os.path
import numpy as np
import pandas as pd

In [27]:
Fifa = pd.read_csv("../../data/prep/FIFA18.csv")

In [28]:
Fantasy1415 = pd.read_csv("../../data/prep/Fantasy1415.csv")

In [29]:
Join1415 = pd.merge(Fifa, Fantasy1415, on=['Surname','FirstInitial'])

In [30]:
Join1415.drop(Join1415.columns[[0]], axis=1, inplace=True)

In [31]:
Join1415.head()

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera Division,1985-02-05,185.0,...,0,3,0,2,0,0,0,35,9.885714,Cristiano
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera Division,1987-06-24,170.0,...,1,0,0,1,0,0,0,38,8.973684,Lionel
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2100,25,French Ligue 1,1992-02-05,175.0,...,4,9,0,0,0,0,0,34,5.558824,Neymar
3,176580,L. Suarez,Luis Suarez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera Division,1987-01-24,182.0,...,2,11,0,0,0,0,0,28,6.035714,Luis
4,225591,L. Suarez,Leonardo Suarez,Villarreal CF,https://cdn.sofifa.org/18/teams/483.png,1673,21,Spanish Primera Division,1996-03-30,167.0,...,2,11,0,0,0,0,0,28,6.035714,Luis


In [32]:
i = 0
array = []

while i < len(Join1415):
    if Join1415['Firstname'][i] in Join1415['name'][i] or Join1415['Firstname'][i] in Join1415['full_name'][i]:
        array.append(True)
        i+=1
    else:
        array.append(False)
        i+=1

In [33]:
array = np.array(array)

In [34]:
Join1415 = Join1415[array]

In [35]:
Error1415 = Join1415[(Join1415['Players'].isin(Join1415['Players'][Join1415['Players'].duplicated()]) | Join1415['full_name'].isin(Join1415['full_name'][Join1415['full_name'].duplicated()]))]
Error1415.head()

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
45,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,0,0,0,0,55,28,1,34,2.088235,Marcelo
46,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,1,6,0,0,31,11,0,33,3.545455,Marcelo
47,180334,Marcelo,Marcelo Antonio Guedes Filho,Olympique Lyonnais,https://cdn.sofifa.org/18/teams/66.png,1789,30,French Ligue 1,1987-05-20,191.0,...,0,0,0,0,55,28,1,34,2.088235,Marcelo
48,180334,Marcelo,Marcelo Antonio Guedes Filho,Olympique Lyonnais,https://cdn.sofifa.org/18/teams/66.png,1789,30,French Ligue 1,1987-05-20,191.0,...,1,6,0,0,31,11,0,33,3.545455,Marcelo
49,205340,Marcelo,Marcelo dos Santos Ferreira,Rio Ave FC,https://cdn.sofifa.org/18/teams/744.png,1625,27,Portuguese Primeira Liga,1989-07-27,182.0,...,0,0,0,0,55,28,1,34,2.088235,Marcelo


In [36]:
leagues = ['German Bundesliga','Spanish Primera Division','English Premier League','French Ligue 1','Italian Serie A']
TopLeagues1415 = Error1415[(Error1415.league.str.contains('|'.join(leagues)) == True) | Error1415['league'].isnull()]

In [37]:
Delete1415 = Error1415[Error1415.league.str.contains('|'.join(leagues)) == False]
Delete1415.head()

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
49,205340,Marcelo,Marcelo dos Santos Ferreira,Rio Ave FC,https://cdn.sofifa.org/18/teams/744.png,1625,27,Portuguese Primeira Liga,1989-07-27,182.0,...,0,0,0,0,55,28,1,34,2.088235,Marcelo
50,205340,Marcelo,Marcelo dos Santos Ferreira,Rio Ave FC,https://cdn.sofifa.org/18/teams/744.png,1625,27,Portuguese Primeira Liga,1989-07-27,182.0,...,1,6,0,0,31,11,0,33,3.545455,Marcelo
57,214378,D. Silva,David Silva,CD Los Millionarios Bogota,https://cdn.sofifa.org/18/teams/101105.png,1939,30,Colombian Primera A,1986-12-13,175.0,...,0,15,0,0,0,0,0,32,4.281250,David
81,192009,J. Rodriguez,Jose Antonio Rodriguez,Club Tijuana,https://cdn.sofifa.org/18/teams/111678.png,1180,24,Mexican Liga MX,1992-07-04,185.0,...,8,11,0,0,0,0,0,25,2.000000,Jose
84,140038,J. Rodriguez,Juan Pablo Rodriguez,Monarcas Morelia,https://cdn.sofifa.org/18/teams/1028.png,1852,37,Mexican Liga MX,1979-08-07,166.0,...,2,7,0,0,0,0,0,34,1.911765,Juan


In [38]:
Fix1415 = TopLeagues1415[(TopLeagues1415['Players'].isin(TopLeagues1415['Players'][TopLeagues1415['Players'].duplicated()]) | TopLeagues1415['full_name'].isin(TopLeagues1415['full_name'][TopLeagues1415['full_name'].duplicated()]))]
Fix1415.head()

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
45,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,0,0,0,0,55,28,1,34,2.088235,Marcelo
46,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,1,6,0,0,31,11,0,33,3.545455,Marcelo
47,180334,Marcelo,Marcelo Antonio Guedes Filho,Olympique Lyonnais,https://cdn.sofifa.org/18/teams/66.png,1789,30,French Ligue 1,1987-05-20,191.0,...,0,0,0,0,55,28,1,34,2.088235,Marcelo
48,180334,Marcelo,Marcelo Antonio Guedes Filho,Olympique Lyonnais,https://cdn.sofifa.org/18/teams/66.png,1789,30,French Ligue 1,1987-05-20,191.0,...,1,6,0,0,31,11,0,33,3.545455,Marcelo
196,209331,M. Salah,Mohamed Salah,Liverpool,https://cdn.sofifa.org/18/teams/9.png,2032,25,English Premier League,1992-06-15,175.0,...,3,0,0,0,0,0,0,3,1.000000,Mohamed


In [39]:
Join1415 = Join1415[(~(Join1415.full_name.isin(Delete1415.full_name) & Join1415.birth_date.isin(Delete1415.birth_date)))]
Join1415 = Join1415[(~(Join1415.full_name.isin(Fix1415.full_name) & Join1415.birth_date.isin(Fix1415.birth_date)))]

In [40]:
Fixed1415 = Fix1415[Fix1415['full_name'] == Fix1415['Players']]
Fixed1415.head()

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
196,209331,M. Salah,Mohamed Salah,Liverpool,https://cdn.sofifa.org/18/teams/9.png,2032,25,English Premier League,1992-06-15,175.0,...,3,0,0,0,0,0,0,3,1.000000,Mohamed
197,209331,M. Salah,Mohamed Salah,Liverpool,https://cdn.sofifa.org/18/teams/9.png,2032,25,English Premier League,1992-06-15,175.0,...,6,7,0,0,0,0,0,16,4.062500,Mohamed
215,189358,S. Kagawa,Shinji Kagawa,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,1874,28,German Bundesliga,1989-03-17,175.0,...,0,0,0,0,0,0,0,0,0.000000,Shinji
216,189358,S. Kagawa,Shinji Kagawa,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,1874,28,German Bundesliga,1989-03-17,175.0,...,5,16,0,0,0,0,0,28,3.178571,Shinji
322,193348,X. Shaqiri,Xherdan Shaqiri,Stoke City,https://cdn.sofifa.org/18/teams/1806.png,2073,25,English Premier League,1991-10-10,169.0,...,6,2,0,0,0,0,0,9,2.555556,Xherdan


In [41]:
Fixed1415 = Fixed1415[(Fixed1415['Players'].isin(Fixed1415['Players'][Fixed1415['Players'].duplicated()]) | Fixed1415['full_name'].isin(Fixed1415['full_name'][Fixed1415['full_name'].duplicated()]))]
Fixed1415.head()

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
196,209331,M. Salah,Mohamed Salah,Liverpool,https://cdn.sofifa.org/18/teams/9.png,2032,25,English Premier League,1992-06-15,175.0,...,3,0,0,0,0,0,0,3,1.000000,Mohamed
197,209331,M. Salah,Mohamed Salah,Liverpool,https://cdn.sofifa.org/18/teams/9.png,2032,25,English Premier League,1992-06-15,175.0,...,6,7,0,0,0,0,0,16,4.062500,Mohamed
215,189358,S. Kagawa,Shinji Kagawa,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,1874,28,German Bundesliga,1989-03-17,175.0,...,0,0,0,0,0,0,0,0,0.000000,Shinji
216,189358,S. Kagawa,Shinji Kagawa,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,1874,28,German Bundesliga,1989-03-17,175.0,...,5,16,0,0,0,0,0,28,3.178571,Shinji
322,193348,X. Shaqiri,Xherdan Shaqiri,Stoke City,https://cdn.sofifa.org/18/teams/1806.png,2073,25,English Premier League,1991-10-10,169.0,...,6,2,0,0,0,0,0,9,2.555556,Xherdan


In [42]:
Fixed1415[Fixed1415.duplicated(['Players', 'Clubs', 'Points'])]

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
375,174379,G. Castro,Gonzalo Castro,Malaga CF,https://cdn.sofifa.org/18/teams/573.png,1878,32,Spanish Primera Division,1984-09-14,178.0,...,3,2,0,0,26,14,0,22,4.0,Gonzalo


In [43]:
Fixed1415 = Fixed1415[~Fixed1415.duplicated(['Players', 'Clubs', 'Points'])]
Fixed1415.head()

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
196,209331,M. Salah,Mohamed Salah,Liverpool,https://cdn.sofifa.org/18/teams/9.png,2032,25,English Premier League,1992-06-15,175.0,...,3,0,0,0,0,0,0,3,1.000000,Mohamed
197,209331,M. Salah,Mohamed Salah,Liverpool,https://cdn.sofifa.org/18/teams/9.png,2032,25,English Premier League,1992-06-15,175.0,...,6,7,0,0,0,0,0,16,4.062500,Mohamed
215,189358,S. Kagawa,Shinji Kagawa,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,1874,28,German Bundesliga,1989-03-17,175.0,...,0,0,0,0,0,0,0,0,0.000000,Shinji
216,189358,S. Kagawa,Shinji Kagawa,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,1874,28,German Bundesliga,1989-03-17,175.0,...,5,16,0,0,0,0,0,28,3.178571,Shinji
322,193348,X. Shaqiri,Xherdan Shaqiri,Stoke City,https://cdn.sofifa.org/18/teams/1806.png,2073,25,English Premier League,1991-10-10,169.0,...,6,2,0,0,0,0,0,9,2.555556,Xherdan


In [44]:
d = {
 'ID' : 'first',
 'club_logo':'first',
 'special':'first',
 'age':'first',
 'league':'first',
 'birth_date':'first',
 'height_cm':'first',
 'weight_kg':'first',
 'body_type':'first',
 'real_face':'first',
 'flag':'first',
 'nationality':'first',
 'photo':'first',
 'eur_value':'first',
 'eur_wage':'first',
 'eur_release_clause':'first',
 'overall':'first',
 'potential':'first',
 'pac':'first',
 'sho':'first',
 'pas':'first',
 'dri':'first',
 'def':'first',
 'phy':'first',
 'international_reputation':'first',
 'skill_moves':'first',
 'weak_foot':'first',
 'work_rate_att':'first',
 'work_rate_def':'first',
 'preferred_foot':'first',
 'crossing':'first',
 'finishing':'first',
 'heading_accuracy':'first',
 'short_passing':'first',
 'volleys':'first',
 'dribbling':'first',
 'curve':'first',
 'free_kick_accuracy':'first',
 'long_passing':'first',
 'ball_control':'first',
 'acceleration':'first',
 'sprint_speed':'first',
 'agility':'first',
 'reactions':'first',
 'balance':'first',
 'shot_power':'first',
 'jumping':'first',
 'stamina':'first',
 'strength':'first',
 'long_shots':'first',
 'aggression':'first',
 'interceptions':'first',
 'positioning':'first',
 'vision':'first',
 'penalties':'first',
 'composure':'first',
 'marking':'first',
 'standing_tackle':'first',
 'sliding_tackle':'first',
 'gk_diving':'first',
 'gk_handling':'first',
 'gk_kicking':'first',
 'gk_positioning':'first',
 'gk_reflexes':'first',
 'rs':'first',
 'rw':'first',
 'rf':'first',
 'ram':'first',
 'rcm':'first',
 'rm':'first',
 'rdm':'first',
 'rcb':'first',
 'rb':'first',
 'rwb':'first',
 'st':'first',
 'lw':'first',
 'cf':'first',
 'cam':'first',
 'cm':'first',
 'lm':'first',
 'cdm':'first',
 'cb':'first',
 'lb':'first',
 'lwb':'first',
 'ls':'first',
 'lf':'first',
 'lam':'first',
 'lcm':'first',
 'ldm':'first',
 'lcb':'first',
 'gk':'first',
 '1_on_1_rush_trait':'first',
 'acrobatic_clearance_trait':'first',
 'argues_with_officials_trait':'first',
 'avoids_using_weaker_foot_trait':'first',
 'backs_into_player_trait':'first',
 'bicycle_kicks_trait':'first',
 'cautious_with_crosses_trait':'first',
 'chip_shot_trait':'first',
 'chipped_penalty_trait':'first',
 'comes_for_crosses_trait':'first',
 'corner_specialist_trait':'first',
 'diver_trait':'first',
 'dives_into_tackles_trait':'first',
 'diving_header_trait':'first',
 'driven_pass_trait':'first',
 'early_crosser_trait':'first',
 "fan's_favourite_trait":'first',
 'fancy_flicks_trait':'first',
 'finesse_shot_trait':'first',
 'flair_trait':'first',
 'flair_passes_trait':'first',
 'gk_flat_kick_trait':'first',
 'gk_long_throw_trait':'first',
 'gk_up_for_corners_trait':'first',
 'giant_throw_in_trait':'first',
 'inflexible_trait':'first',
 'injury_free_trait':'first',
 'injury_prone_trait':'first',
 'leadership_trait':'first',
 'long_passer_trait':'first',
 'long_shot_taker_trait':'first',
 'long_throw_in_trait':'first',
 'one_club_player_trait':'first',
 'outside_foot_shot_trait':'first',
 'playmaker_trait':'first',
 'power_free_kick_trait':'first',
 'power_header_trait':'first',
 'puncher_trait':'first',
 'rushes_out_of_goal_trait':'first',
 'saves_with_feet_trait':'first',
 'second_wind_trait':'first',
 'selfish_trait':'first',
 'skilled_dribbling_trait':'first',
 'stutter_penalty_trait':'first',
 'swerve_pass_trait':'first',
 'takes_finesse_free_kicks_trait':'first',
 'target_forward_trait':'first',
 'team_player_trait':'first',
 'technical_dribbler_trait':'first',
 'tries_to_beat_defensive_line_trait':'first',
 'poacher_speciality':'first',
 'speedster_speciality':'first',
 'aerial_threat_speciality':'first',
 'dribbler_speciality':'first',
 'playmaker_speciality':'first',
 'engine_speciality':'first',
 'distance_shooter_speciality':'first',
 'crosser_speciality':'first',
 'free_kick_specialist_speciality':'first',
 'tackling_speciality':'first',
 'tactician_speciality':'first',
 'acrobat_speciality':'first',
 'strength_speciality':'first',
 'clinical_finisher_speciality':'first',
 'prefers_rs':'first',
 'prefers_rw':'first',
 'prefers_rf':'first',
 'prefers_ram':'first',
 'prefers_rcm':'first',
 'prefers_rm':'first',
 'prefers_rdm':'first',
 'prefers_rcb':'first',
 'prefers_rb':'first',
 'prefers_rwb':'first',
 'prefers_st':'first',
 'prefers_lw':'first',
 'prefers_cf':'first',
 'prefers_cam':'first',
 'prefers_cm':'first',
 'prefers_lm':'first',
 'prefers_cdm':'first',
 'prefers_cb':'first',
 'prefers_lb':'first',
 'prefers_lwb':'first',
 'prefers_ls':'first',
 'prefers_lf':'first',
 'prefers_lam':'first',
 'prefers_lcm':'first',
 'prefers_ldm':'first',
 'prefers_lcb':'first',
 'prefers_gk':'first',
 'Players':'first',
 'Position':'first',
 'Clubs':'first',
 'Points':'sum',  
 'Start':'sum', 
 'Sub':'sum', 
 'club':'last',
 'Position':'first',
 'Start':'sum',
 'Goals':'sum',
 'A':'sum',
 'CS':'sum',
 'CS part':'sum',
 'Yellow':'sum',
 'Red':'sum',
 'Sub':'sum',
 'CO':'sum',
 'Off':'sum',
 'Pen SV':'sum',
 'Pen M':'sum',
 'Goals conceded':'sum',
 'Conceded 1+':'sum',
 'OG':'sum',
 'Firstname':'first',
 'Apps':'sum',
 'Form': 'mean'
}

In [45]:
Fixed1415 = (Fixed1415.groupby(['name','full_name'], sort=False, as_index=False).agg(d).reindex(columns=Fix1415.columns))
Fixed1415.head()

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,209331,M. Salah,Mohamed Salah,Liverpool,https://cdn.sofifa.org/18/teams/9.png,2032,25,English Premier League,1992-06-15,175.0,...,9,7,0,0,0,0,0,19,2.531250,Mohamed
1,189358,S. Kagawa,Shinji Kagawa,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,1874,28,German Bundesliga,1989-03-17,175.0,...,5,16,0,0,0,0,0,28,1.589286,Shinji
2,193348,X. Shaqiri,Xherdan Shaqiri,Stoke City,https://cdn.sofifa.org/18/teams/1806.png,2073,25,English Premier League,1991-10-10,169.0,...,13,7,0,0,0,0,0,24,2.311111,Xherdan
3,193082,J. Cuadrado,Juan Cuadrado,Juventus,https://cdn.sofifa.org/18/teams/45.png,2111,29,Italian Serie A,1988-05-26,179.0,...,8,12,0,0,0,0,0,29,2.377451,Juan
4,178224,J. Hernandez,Javier Hernandez,West Ham United,https://cdn.sofifa.org/18/teams/19.png,1917,29,English Premier League,1988-06-01,175.0,...,17,3,0,0,0,0,0,25,2.520833,Javier


In [46]:
Fixed1415[['full_name','club','Players','Clubs','Points']]

,full_name,club,Players,Clubs,Points
0,Mohamed Salah,Liverpool,Mohamed Salah,CHEL,68
1,Shinji Kagawa,Borussia Dortmund,Shinji Kagawa,MUFC,89
2,Xherdan Shaqiri,Stoke City,Xherdan Shaqiri,BM1,54
3,Juan Cuadrado,Juventus,Juan Cuadrado,CHEL,75
4,Javier Hernandez,West Ham United,Javier Hernandez,MUFC,75
5,Gonzalo Castro,Borussia Dortmund,Gonzalo Castro,BL1,88
6,Andre Schurrle,Borussia Dortmund,Andre Schurrle,CHEL,63
7,Salomon Kalou,Hertha BSC Berlin,Salomon Kalou,HB1,78
8,Milan Badelj,Fiorentina,Milan Badelj,HSV,51
9,Benjamin Stambouli,FC Schalke 04,Benjamin Stambouli,SPURS,16


In [47]:
Joined1415 = pd.concat([Join1415,Fixed1415])
Joined1415 = Joined1415.reset_index()

In [48]:
Joined1415.drop(Joined1415.columns[[0]], axis=1, inplace=True)

In [49]:
Joined1415.head()

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera Division,1985-02-05,185.0,...,0,3,0,2,0,0,0,35,9.885714,Cristiano
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera Division,1987-06-24,170.0,...,1,0,0,1,0,0,0,38,8.973684,Lionel
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2100,25,French Ligue 1,1992-02-05,175.0,...,4,9,0,0,0,0,0,34,5.558824,Neymar
3,176580,L. Suarez,Luis Suarez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera Division,1987-01-24,182.0,...,2,11,0,0,0,0,0,28,6.035714,Luis
4,167495,M. Neuer,Manuel Neuer,FC Bayern Munich,https://cdn.sofifa.org/18/teams/21.png,1493,31,German Bundesliga,1986-03-27,193.0,...,1,0,1,0,18,6,0,32,4.406250,Manuel


In [50]:
Joined1415.to_csv('../../data/prep/Joined1415.csv')